In [15]:
#Start Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Shoplytics Market Basket Analysis") \
    .config("spark.driver.extraJavaOptions", "-Xss4m") \
    .getOrCreate()

print("✅ Spark session restarted.")


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
#Load Cleaned Amazon CSV
#Load the CSV you created in the previous notebook
df_amazon = spark.read.csv("../data/amazon_reviews/amazon_cleaned_electronics.csv", header=True, inferSchema=True)

df_amazon.select("product_id", "product_title", "review_date").show(5)


+----------+--------------------+-----------+
|product_id|       product_title|review_date|
+----------+--------------------+-----------+
|B00428R89M|yoomall 5M Antenn...| 2015-08-31|
|B000068O48|"Hosa GPM-103 3.5...| 2015-08-31|
|B000GGKOG8|Channel Master Ti...| 2015-08-31|
|B000NU4OTA|LIMTECH Wall char...| 2015-08-31|
|B00JOQIO6S|Skullcandy Air Ra...| 2015-08-31|
+----------+--------------------+-----------+
only showing top 5 rows



In [12]:
#Create Transaction Baskets

from pyspark.sql.functions import collect_set

basket_df = df_amazon.groupBy("product_category", "review_date") \
    .agg(collect_set("product_title").alias("items")) \
    .filter("size(items) > 1 AND size(items) < 20")


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
#Run FP-Growth
from pyspark.ml.fpm import FPGrowth

fp_growth = FPGrowth(itemsCol="items", minSupport=0.002, minConfidence=0.3)
model = fp_growth.fit(basket_df)

frequent_itemsets = model.freqItemsets
association_rules = model.associationRules

frequent_itemsets.show(truncate=False)
association_rules.show(truncate=False)



25/04/13 22:24:14 WARN DAGScheduler: Broadcasting large task binary with size 1018.8 KiB
25/04/13 22:24:16 ERROR Executor: Exception in task 1.0 in stage 28.0 (TID 106)]
java.lang.StackOverflowError
	at java.base/java.lang.reflect.InvocationTargetException.<init>(InvocationTargetException.java:67)
	at jdk.internal.reflect.GeneratedMethodAccessor7.invoke(Unknown Source)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:569)
	at java.base/java.io.ObjectStreamClass.invokeWriteObject(ObjectStreamClass.java:1070)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1516)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1438)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1181)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1572)
	at java.base/jav

Py4JError: py4j does not exist in the JVM